# Day 14 "Extended Polymerization"

## Part 1

### Problem

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

    NNCB

    CH -> B
    HH -> N
    CB -> H
    NH -> C
    HB -> C
    HC -> B
    HN -> C
    NN -> C
    BH -> H
    NC -> B
    NB -> B
    BN -> B
    BB -> N
    BC -> B
    CC -> N
    CN -> C

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

* The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
* The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
* The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

    Template:     NNCB
    After step 1: NCNBCHB
    After step 2: NBCCNBBBCBHCB
    After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
    After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 191 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

### Setup

Parse and style.

In [25]:
from utils import *

_input = initDay('day14')
_sample = getMarkdown('For example')

def parse(text):
    items = re.findall(r'\w+', text)
    rules = dict([(s, (s[0]+d, d+s[1])) for (s, d) in zip(items[1::2], items[2::2])])
    pairs = dict([(pair, 0) for pair in rules.keys()])
    for i in range(len(items[0])-1):
        pairs[items[0][i:i+2]] += 1

    return items[0], pairs, rules


### Solution

The sample walks through a process of building up a chain, but there is no reason to do this. The algorithm works on pairs, and the only thing relevant about the entire chain is that there is a begin and end which must be accounted for.

The algorithm is simple:

1. Break the input into pairs, recording how many times each occurs
2. For each iteration, go through each pair and find the relevant rule to get the two new pairs to produce
3. Increment the count of each new pair and reduce the count of the old pair by the size of the old pair
4. Do some simple math to get the final answer

Working in pairs means that we're duplicating each element once, so we cut in half for the answer. Note that the first and last input elements aren't doubled because there is no "other" side to them, so add them into the doubling to make them consistent.

In [26]:
from collections import defaultdict

def solve(text, steps):
    input, pairs, rules = parse(text)
    for _ in range(steps):
        prev = dict(pairs)
        for pair, count in prev.items():
            dst = rules[pair]
            pairs[dst[0]] += count
            pairs[dst[1]] += count
            pairs[pair] -= count

    counts = defaultdict(lambda:0)
    for pair, count in pairs.items():
        counts[pair[0]] += count
        counts[pair[1]] += count

    # fix first/last to be doubled like the other pairs
    counts[input[0]] += 1
    counts[input[-1]] += 1

    return max(counts.values())//2 - min(counts.values())//2

check(solve(_sample, 10), 1588)
check1(solve(_input, 10))

Part 1 Result: 4244


## Part 2

### Problem

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

### Solution

Solution is the same, just more iterations.

In [27]:
check(solve(_sample, 40), 2188189693529)
check2(solve(_input, 40))

Part 2 Result: 4807056953866
